In [35]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import numpy as np
import json

In [144]:
class SeparableConv2D(nn.Module):
    '''Separable convolution'''
    def __init__(self, in_channels, out_channels, stride=1):
        super(SeparableConv2D, self).__init__()
        self.dw_conv = nn.Sequential(
            nn.Conv2d(in_channels, in_channels, kernel_size=3, stride=stride, padding=1, groups=in_channels, bias=False),
            nn.BatchNorm2d(in_channels),
            nn.ReLU(inplace=False),
        )
        self.pw_conv = nn.Sequential(
            nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=1, padding=0, bias=False),
            nn.BatchNorm2d(out_channels),
            nn.ReLU(inplace=False),
        )

    def forward(self, x):
        x = self.dw_conv(x)
        x = self.pw_conv(x)
        return x

In [233]:
input = torch.randn((1, 3, 5, 5))
model = SeparableConv2D(3, 3)

In [258]:
def DepthwiseConvInt(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    Input = [[[str(input[i][j][k] % p) for k in range(nChannels)] for j in range(nCols)] for i in range(nRows)]
    Weights = [[[str(weights[i][j][k] % p) for k in range(nChannels)] for j in range(kernelSize)] for i in range(kernelSize)]
    Bias = [str(bias[i] % p) for i in range(nFilters)]
    out = [[[0 for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    remainder = [[[None for _ in range(nFilters)] for _ in range(outCols)] for _ in range(outRows)]
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row][col][channel] += input[row*strides+x][col*strides+y][channel] * weights[x][y][channel]
                        
                out[row][col][channel] += bias[channel]
                remainder[row][col][channel] = str(out[row][col][channel] % n)
                out[row][col][channel] = str((out[row][col][channel] // n) % p)
                
    return Input, Weights, Bias, out, remainder
    
def DepthwiseConv(nRows, nCols, nChannels, nFilters, kernelSize, strides, n, input, weights, bias):
    assert(nFilters % nChannels == 0)
    outRows = (nRows - kernelSize)//strides + 1
    outCols = (nCols - kernelSize)//strides + 1
    
    out = np.zeros((outRows, outCols, nFilters))
    remainder = np.zeros((outRows, outCols, nFilters))
    
    for row in range(outRows):
        for col in range(outCols):
            for channel in range(nChannels):
                for x in range(kernelSize):
                    for y in range(kernelSize):
                        out[row, col, channel] += input[row*strides+x, col*strides+y, channel] * weights[x, y, channel]
                
                out[row][col][channel] += bias[channel]
                remainder[row][col][channel] = out[row][col][channel] % n
                out[row][col][channel] = out[row][col][channel] / n
                            
    return out, remainder

weights = model.dw_conv[0].weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()
# input = torch.randn((1, 8, 32, 32))

expected = model.dw_conv[0](input).detach().numpy()

# # Converting to H x W x C
padded = F.pad(input, (1,1,1,1), "constant", 0) # Padding for convolution with "same" configuration
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

actual, rem = DepthwiseConv(7, 7, 3, 3, 3, 1, 1, padded, weights, bias)
expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual, atol=0.00001))

In [178]:
EXPONENT = 8
    
weights = model.dw_conv[0].weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()
# input = torch.randn((1, 8, 32, 32))

expected = model.dw_conv[0](input).detach().numpy()

# # Converting to H x W x C
padded = F.pad(input, (1,1,1,1), "constant", 0)
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

quantized_image = padded * 10**EXPONENT
quantized_weights = weights * 10**EXPONENT

actual, rem = DepthwiseConv(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round(), quantized_weights.round(), bias)
actual = actual / 10**(EXPONENT)

expected = expected.squeeze().transpose((1, 2, 0))

assert(np.allclose(expected, actual, atol=0.00001))

In [257]:
weights = model.dw_conv[0].weight.squeeze().detach().numpy()
bias = torch.zeros(weights.shape[0]).numpy()

padded = F.pad(input, (1,1,1,1), "constant", 0)
padded = padded.squeeze().numpy().transpose((1, 2, 0))
weights = weights.transpose((1, 2, 0))

quantized_image = padded * 10**EXPONENT
quantized_weights = weights * 10**EXPONENT

q_input, q_weights, bias, actual, rem = DepthwiseConvInt(7, 7, 3, 3, 3, 1, 10**EXPONENT, quantized_image.round().astype(int), quantized_weights.round().astype(int), bias.astype(int))

input_json_path = "depthwiseConv2D_input.json"
with open(input_json_path, "w") as input_file:
    json.dump({"in": q_input,
               "weights": q_weights,
               "remainder": rem,
               "out": actual,
               "bias": bias,
              },
              input_file)